In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
import json
from playwright.async_api import async_playwright
import time
from collections import Counter

In [2]:
df = pd.read_csv("parliament questions.csv")  
pd.set_option('display.max_columns', None)

In [3]:
df.head(10) 

,Title of debate,Date,Members,Classification,Year
0,Promotion of Plantation,04/03/23,Upendra Singh Rawat,Forestry and plantation,2023
1,Environmental Clearance to Construction of Add...,04/03/23,Rajkumar Chahar,Environmental clearances,2023
2,Dumping Garbage in River Banks,04/03/23,Dilip SaikiaNaranbhai KachhadiaRanjeetsingh Na...,Pollution,2023
3,Setting up of Zoos,04/03/23,Amol Ramsing KolheVijay Kumar,Animal and wildlife,2023
4,Relocation of Elephants,04/03/23,Prajwal Revanna,Animal and wildlife,2023
5,Hazardous Waste,04/03/23,Kinjarapu Ram Mohan NaiduKirit Premjibhai Solanki,Pollution,2023
6,Emission Inventory Report,04/03/23,Satyadev Pachauri,Pollution,2023
7,Afforestation Programmes,04/03/23,Rajveer Diler,Forestry and plantation,2023
8,Utilisation of Funds,04/03/23,Shashi Tharoor,Others,2023
9,Allocation of Funds under CAMPA,04/03/23,Srinivas KesineniMaddila Gurumoorthy,Forestry and plantation,2023


In [4]:
df['Title of debate'] = df['Title of debate'].str.lower()
df

,Title of debate,Date,Members,Classification,Year
0,promotion of plantation,04/03/23,Upendra Singh Rawat,Forestry and plantation,2023
1,environmental clearance to construction of add...,04/03/23,Rajkumar Chahar,Environmental clearances,2023
2,dumping garbage in river banks,04/03/23,Dilip SaikiaNaranbhai KachhadiaRanjeetsingh Na...,Pollution,2023
3,setting up of zoos,04/03/23,Amol Ramsing KolheVijay Kumar,Animal and wildlife,2023
4,relocation of elephants,04/03/23,Prajwal Revanna,Animal and wildlife,2023
...,...,...,...,...,...
4349,encroachment on forest land,02/10/14,Sultan Ahmed,NaN,2014
4350,conservation of bio diversity,02/10/14,Shruti Choudhury,NaN,2014
4351,environment protection act,02/10/14,Ratan Singh,NaN,2014
4352,streamlining of clearance process,02/10/14,Maulana Badruddin AjmalP.K. BijuN.S.V. Chittha...,NaN,2014


In [246]:
df.loc[df['Title of debate'].str.contains('promotion of plantation|pine needles|depletion of natural resources|vegetation cover|red sanders|environmentally sensitive zone|cza|nagar van|nature|sacred groves|seagrass|green belt areas|grassland|ecological sensitive areas|jungle|forest|plantation|tree|afforestation|agro forestry|bio diversity|bio-diversity|buffer|species|nilgai'), 'category']='Forestry' 
df.loc[df['Title of debate'].str.contains('climate goals|foreign weeds|habitat loss|rhino|asiatic golden cats|cobra|painted storks|turtle|human-wildlife conflict|tigress|bustard|rhinoceros|peacock|redd|juliflora|leopard|gian|national park|wolf|biodiversity|bird|conflict|wildlife|boar|snake|vulture|poaching|lion|sloth bear|loss of wild-life|goals achieved under indcs|dolphin|tiger|zoo|elephant|animal|cheetah|snail|reserve|falcon|wildlife sancturies|sanctuaries|fencing|buck| shark|sanctuary|monkey|gian|captive bears'), 'category']='Wildlife' 
df.loc[df['Title of debate'].str.contains('united nations environment programme|initiatives under sawen|cop-21 resolutions|cop-24 of unfccc|swachh-nirmal tat abhiyan|kyoto protocol|paris|swachh-nirmal tat abhiyaan|overhauling of environmental framework|environmental governance architecture|environmental certification|phasing out fossil fuel|environment regulatory mechanism|crz|pushing of industrial interest by developed nations|setting up|kyoto protocol targets|ban|swach-nirmal tat abhiyan|nccf|national green corps programme|cop26|glasgow summit|steps to control desertification|ban on the use of asbestos|rule|plan|eeat|act|yojana|adoption|norms|agreement|campa|law|epa|scheme|paris agreement|project|policy|blue flag|itbp|adaptation fund|fund| finance|parivesh portal|cop 26|environmental protection|water purifier|decarbonize the industrial sector|mission|prevent erosion in coastal areas|state coastal zone management authority|funds received/utilised to fight climate change|conference on climate change|climate change mitigation actions|research on climate change|carbon-neutral village|financial support to brick kilns|lifestyle for environment|performance-based grants|beautification of beaches|ncap|guidelines|measures|caqm|conference of parties|ban on manja|cross dependency initiative|ban on generator|dilution of tasks at climate talks'), 'category']='Climate policy'
df.loc[df['Title of debate'].str.contains('lowering soil & water ph|fly ash|burning of biomass|straw burning|vehicle density|polythene bags|toxic air|crop residue|tds|diesel|particulate matter|generators|discharge|smog|aqi|buddha nalla|river|yamuna|clean air|pollution|garbage|river|polluting|purity|stations|pollutants|purification|purifier|quality|contamination|alien|cracker|klin|emission|ncap cities|methane|coal|polluted|dying units|stubble burning|urban agglomeration|non-attainment cities|power station|obsolescence|burning of paddy straw'), 'category']='Pollution'
df.loc[df['Title of debate'].str.contains('no objection certificate|mining|clearance|mine|vista|environment impact assessment|auroville'), 'category']='Env Clearances'
df.loc[df['Title of debate'].str.contains('marine life|environmentally vulnerable areas|relocation|carbon footprint|ground water|wildfire|use of fossil fuels|extreme weather events|heavy rain|environmental crisis|death|glaciers|damage|volatility in climatic conditions|destruction|cyclone|mercury|biosphere|calamity|melt|assessment|ozone|depletion of natural capital assets|coral reef|flood|oil spill|issue|sunderban|threat|climatic change|temperature|segregation of carbon|cleanliness|degradation|ecology|hotspot|climate refugees|mangrove|abnormal climate patterns|environmental compensation charge|global warming|western ghats|desertification|climate crisis|eia|black carbon|carbon sink|monsoon|migration|emergency|wetland|climate vulnerability|sea level|landslide|extremes|impact of climate change on odisha|sea erosion|erosion of sea beaches|state disaster management|ecological sensitive zones in the himalayas|land subsidence|displacement|immediate response|marsh lands|green house gases|disasters|massive disaster of flora and fauna'), 'category']='Climate impact'
df.loc[df['Title of debate'].str.contains('plastic|bio medical|bio-medical|waste|sewage|treatment|board|pet'), 'category']='Waste'
df.loc[df['Title of debate'].str.contains('closing of tanneries|rainwater harvesting|protection|corporate environmental responsibility|regeneration of lakes|png|environment management capacity|carbon stock|preservation|beautification|eco|electric vehicles|sustainable|green|development of green belt|ramsar|rehabilitation|restoration|revival|eco-friendly beaches|extended producers responsibility|extended producer liability|esz|conservation|circular economy|cng|replacement of diesel based generators|green gross domestic product|technology assessment cells|incentivise recycling|epi|segregator packaging indicator|reusable, recyclable or compostable packaging|no-development zones|star rating system|extended producer responsibility|bio-digester units|hydrogen fuel|development of clean technology'), 'category']='Conservation'
df.loc[df['Title of debate'].str.contains('killing of tuskers|smuggling|encroachment|penalty|violation|unauthorised|offences|crime|illegal|environmental cases|ivory|encroachment of land|public hearing on environmental issues|greenwashing mechanism|arrest of poachers'), 'category']='Crimes'
df.loc[df['Title of debate'].str.contains('regional office|shortage|vacant|spcb|cpcb|vacancy|tribunal'), 'category']='Infrastructure'
df.loc[df['Title of debate'].str.contains('electro-magnetic radiation|cow slaughter|traditional medical knowledge|genetically modified|solar air purifying unit|passenger ropewaysproduction of ethanol|genetically modified soybean|life on land|chemical accidents|climate research centres|compensation to farmers|integrating environmental concerns|nistaar land|modernisation of saw mills|free prior and informed consent|ngt directive on old vehicles|indian print media|cbipm|environmental education|eviction of farmers|salt pan lands|carbon trading|self certification notification|veterinary hositals|water cess|pending irrigation proposals|bamboo harvesting|brick kilns|proposals from north east|environmental consciousness|proposals for new dams|diversion of land in kutch desert|developmental works in rural areas|recycling battery|magazines|parivesh|replacement of leased vehicles|gyrones|shift in energy production|proper utilisation of paper|tribal|regional offices|port|osmosis|world environment day|index|gm|closure|elite global tag|effects of baghjan oilfield blowout|environment audit report|global budget conference|non-permanent structures at tourist destinations|national conference of environment minister|cpcb|committee to inspect textile units|light detection and ranging based survey|haryali mahotsav|genetic engineering approval committee|private sector in adaptation technologies'), 'category']='Others'






In [247]:
df

,Title of debate,Date,Members,Classification,Year,category
0,promotion of plantation,04/03/23,Upendra Singh Rawat,Forestry and plantation,2023,Climate policy
1,environmental clearance to construction of add...,04/03/23,Rajkumar Chahar,Environmental clearances,2023,Env Clearances
2,dumping garbage in river banks,04/03/23,Dilip SaikiaNaranbhai KachhadiaRanjeetsingh Na...,Pollution,2023,Pollution
3,setting up of zoos,04/03/23,Amol Ramsing KolheVijay Kumar,Animal and wildlife,2023,Climate policy
4,relocation of elephants,04/03/23,Prajwal Revanna,Animal and wildlife,2023,Climate impact
...,...,...,...,...,...,...
4349,encroachment on forest land,02/10/14,Sultan Ahmed,NaN,2014,Crimes
4350,conservation of bio diversity,02/10/14,Shruti Choudhury,NaN,2014,Conservation
4351,environment protection act,02/10/14,Ratan Singh,NaN,2014,Conservation
4352,streamlining of clearance process,02/10/14,Maulana Badruddin AjmalP.K. BijuN.S.V. Chittha...,NaN,2014,Env Clearances


In [249]:
print(df.iloc[0])

Title of debate    promotion of plantation
Date                              04/03/23
Members                Upendra Singh Rawat
Classification     Forestry and plantation
Year                                  2023
category                    Climate policy
Name: 0, dtype: object


In [245]:
df.to_csv('category.csv', encoding='utf-8')
